<img src="./code/ToyNetwork-Palsson.pdf" width="500">

In [54]:
# include -
include("./code/Include.jl");

##### Load the stoichiometric matrix 

In [6]:
stoichiometric_matrix = readdlm("./code/network/Network.csv",',')

8×9 Array{Float64,2}:
 -1.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0
  1.0  -1.0  -1.0   1.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   1.0  -1.0  -1.0   1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0  -1.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0

In [8]:
(number_of_metabolites, number_of_reactions) = size(stoichiometric_matrix)

(8, 9)

##### Compute the metabolite ($A_{m}$) and reaction ($A_{v}$) Adjacency matrices

In [18]:
# Adj matricies are calculated from the binary stoichiometic array -
# all non-zero values = 1.0
binary_stoichiometric_matrix = make_binary_stoichiometric_matrix(stoichiometric_matrix)

8×9 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [19]:
# Metabolite adj matrix -
A_m = binary_stoichiometric_matrix*transpose(binary_stoichiometric_matrix)

8×8 Array{Float64,2}:
 2.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  4.0  1.0  2.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  0.0  0.0  0.0  1.0  0.0
 0.0  2.0  0.0  4.0  2.0  0.0  0.0  0.0
 0.0  0.0  0.0  2.0  3.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0

In [20]:
# Reaction adj matrix -
A_v = transpose(binary_stoichiometric_matrix)*binary_stoichiometric_matrix

9×9 Array{Float64,2}:
 2.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
 1.0  2.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0
 1.0  1.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0
 1.0  1.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  2.0  2.0  0.0  0.0  1.0
 0.0  0.0  1.0  1.0  2.0  2.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  2.0

#### Singular value decomposition (SVD) of the stoichiometric matrix

The [SVD of a matrix](https://www.youtube.com/watch?v=mBcLRGuAFUk) is a method to decompose an array $A$ into three arrays:

\begin{equation}
A = U\Sigma{V}^{T}
\end{equation}

which is the product of rotation $\times$ stretch $\times$ rotation operations. The SVD of the stoichiometric array has been used to get some additional structural insight into a biological reaction network (see the [Palsson study](https://www.ncbi.nlm.nih.gov/pubmed/12900206) or the [Palsson SVD stoichiometric analysis video](https://www.youtube.com/watch?v=8W7RBt_iCYM&feature=youtu.be)).

In simple terms, the SVD of an array pulls the matrix into a set of weight modes, where each mode represents a section of the original matrix, where the weight (importance) of that section is given by the singular values. The original matrix can be reconstructed as the weighted sum of [outer products](https://en.wikipedia.org/wiki/Outer_product):

\begin{equation}
A = \sum_{i=1}^{r}\sigma_{i}\left(u_{i}\otimes{v_{i}^{T}}\right)
\end{equation}

where $r$ denotes the [rank](https://en.wikipedia.org/wiki/System_of_linear_equations) of the matrix $A$. 

In [23]:
(U,S,V) = svd(stoichiometric_matrix);

##### Check accuracy:

In [31]:
error_term = norm(stoichiometric_matrix - U*diagm(S)*transpose(V));

In [32]:
error_term

8.713796557141027e-15

In [58]:
# the U matrix governs metabolite combinations -
U

8×8 Array{Float64,2}:
  0.126327    0.308825   -0.601501     …   0.371748     -0.238715   0.353553
 -0.584093   -0.575166    3.94129e-15      5.55112e-17  -0.0144711  0.353553
  0.126327    0.308825    0.601501        -0.371748     -0.238715   0.353553
  0.690286   -0.257872    1.72085e-15      1.15186e-15   0.212413   0.353553
 -0.380989    0.598011   -4.32987e-15      2.02616e-15   0.400544   0.353553
 -0.0217949  -0.0972011   0.371748     …   0.601501     -0.375857   0.353553
 -0.0217949  -0.0972011  -0.371748        -0.601501     -0.375857   0.353553
  0.0657311  -0.188221    1.83187e-15      3.10862e-15   0.630658   0.353553

In [60]:
# the V matrix governs reaction combinations -
V

9×8 Adjoint{Float64,Array{Float64,2}}:
 -0.27251    -0.43252    0.371748     -0.172859   …   0.371233  -9.81308e-17
  0.27251     0.43252    0.371748      0.172859      -0.371233  -4.4437e-15 
  0.488839    0.155246  -1.16573e-15  -0.308278       0.375604  -0.289121   
 -0.488839   -0.155246   1.22125e-15   0.308278      -0.375604  -0.289121   
 -0.410931    0.418767  -3.67241e-15  -0.0824577      0.31145    0.645297   
  0.410931   -0.418767   3.6169e-15    0.0824577  …  -0.31145    0.645297   
  0.0568182   0.198661  -0.601501      0.50309        0.227037  -9.81308e-17
 -0.0568182  -0.198661  -0.601501     -0.50309       -0.227037  -2.42861e-15
  0.171358   -0.384688   3.71925e-15   0.479972       0.38095    2.27596e-15

In [62]:
# The S matrix is a diagonal matrix that contains the singular values (weights in each mode)
diagm(S)

8×8 Array{Float64,2}:
 2.60695  0.0      0.0      0.0      0.0      0.0       0.0       0.0       
 0.0      2.04381  0.0      0.0      0.0      0.0       0.0       0.0       
 0.0      0.0      1.61803  0.0      0.0      0.0       0.0       0.0       
 0.0      0.0      0.0      1.53088  0.0      0.0       0.0       0.0       
 0.0      0.0      0.0      0.0      1.14812  0.0       0.0       0.0       
 0.0      0.0      0.0      0.0      0.0      0.618034  0.0       0.0       
 0.0      0.0      0.0      0.0      0.0      0.0       0.604051  0.0       
 0.0      0.0      0.0      0.0      0.0      0.0       0.0       4.5905e-17

In [34]:
rank(stoichiometric_matrix)

7

In [36]:
fraction = (S./sum(S))*100.0

8-element Array{Float64,1}:
 25.63401371619153     
 20.096743282436805    
 15.910061691856608    
 15.05308187167751     
 11.289383499874889    
  6.077102803181567    
  5.939613134781095    
  4.513817863863056e-16

In [49]:
FM = fractional_reconstruction_of_stoichiometric_matrix(stoichiometric_matrix, U,S,V);

In [50]:
FM[1]

8×9 Array{Float64,2}:
 -0.0897453   0.0897453   0.160989   -0.160989   …  -0.0187118    0.0564329 
  0.414951   -0.414951   -0.744355    0.744355       0.086517    -0.260926  
 -0.0897453   0.0897453   0.160989   -0.160989      -0.0187118    0.0564329 
 -0.490393    0.490393    0.879686   -0.879686      -0.102247     0.308365  
  0.270662   -0.270662   -0.485525    0.485525       0.0564329   -0.170196  
  0.0154835  -0.0154835  -0.027775    0.027775   …   0.0032283   -0.00973623
  0.0154835  -0.0154835  -0.027775    0.027775       0.0032283   -0.00973623
 -0.0466967   0.0466967   0.0837664  -0.0837664     -0.00973623   0.0293635 

In [51]:
FM[2]

8×9 Array{Float64,2}:
 -0.362744   0.362744   0.258977   …   0.144103   -0.144103   -0.186375 
  0.923393  -0.923393  -0.926852      -0.32005     0.32005     0.191288 
 -0.362744   0.362744   0.258977       0.144103   -0.144103   -0.186375 
 -0.262436   0.262436   0.797865      -0.0024562   0.0024562   0.511112 
 -0.257974   0.257974  -0.29578        0.186375   -0.186375   -0.640371 
  0.101408  -0.101408  -0.0586163  …  -0.0426945   0.0426945   0.0666863
  0.101408  -0.101408  -0.0586163     -0.0426945   0.0426945   0.0666863
  0.119689  -0.119689   0.024045      -0.0666863   0.0666863   0.177349 

In [52]:
FM[end]

8×9 Array{Float64,2}:
 -1.0           8.39606e-16  -3.1225e-16   …  -1.08941e-15  -3.747e-16  
  1.0          -1.0          -1.0              2.37787e-15  -1.23599e-16
  1.13798e-15   1.0          -4.71845e-16     -1.0           1.38778e-17
 -3.1225e-16   -6.59195e-16   1.0              1.09635e-15   2.77556e-17
  1.42941e-15   8.46545e-16   4.30211e-16     -2.51188e-15  -1.0        
 -6.38378e-16  -3.05311e-16  -5.55112e-17  …   5.96745e-16   6.93889e-17
 -6.245e-16    -1.80411e-16  -7.21645e-16      1.0          -1.38778e-16
 -2.498e-16    -2.22045e-16   4.71845e-16     -5.55112e-17   1.0        

##### Is there information in the U and V arrays?

In [72]:
MSV = ["A","B","C","D","E","Ax","Cx","Ex"]
idx_importance = sortperm(abs.(U[:,1]));
[MSV[idx_importance] idx_importance abs.(U[idx_importance,1])]

8×3 Array{Any,2}:
 "Cx"  7  0.0217949
 "Ax"  6  0.0217949
 "Ex"  8  0.0657311
 "A"   1  0.126327 
 "C"   3  0.126327 
 "E"   5  0.380989 
 "B"   2  0.584093 
 "D"   4  0.690286 

In [73]:
RSV = ["v1","v2","v3","v4","v5","v6","q1","q2","q3"]
idx_importance = sortperm(abs.(V[:,1]));
[RSV[idx_importance] idx_importance abs.(V[idx_importance,1])]

9×3 Array{Any,2}:
 "q2"  8  0.0568182
 "q1"  7  0.0568182
 "q3"  9  0.171358 
 "v1"  1  0.27251  
 "v2"  2  0.27251  
 "v5"  5  0.410931 
 "v6"  6  0.410931 
 "v3"  3  0.488839 
 "v4"  4  0.488839 